In [ ]:
%cd /Users/Pracioppo/Desktop/VPTR/VPTR/

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter

from torch import nn
import functools
from torch.nn import init

from pathlib import Path
import random
from datetime import datetime

import scipy.io as sio

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Loading bar
from PIL import Image

from model import GDL, MSELoss, L1Loss
from utils import get_dataloader
from utils import visualize_batch_clips, save_ckpt, load_ckpt, set_seed, AverageMeters, init_loss_dict, write_summary, resume_training
from utils import set_seed
from utils import f_rescale_dataset, f_Residuals, f_reshape_training_data, f_rotate_and_zoom, f_random_crop, f_rotate_and_zoom_all, f_crop_all, f_flip_all, f_augment_dataset2

from utils import KTHDataset, BAIRDataset, MovingMNISTDataset

set_seed(2023)

import argparse

import cv2
from tabulate import tabulate


ckpt_save_dir = Path('/Users/Pracioppo/Desktop//VPTR/GA_Zubens_data//Dyn_ResNetAE_MSEGDLgan_ckpt')
tensorboard_save_dir = Path('/Users/Pracioppo/Desktop/VPTR/GA_Zubens_data/Dyn_ResNetAE_MSEGDLgan_tensorboard')
                            
start_epoch = 0

# resume_ckpt = r"\Users\Pracioppo\Desktop\VPTR\GA_2\GA_full_3chan_gan_ckpt\epoch_20.tar"
resume_ckpt = None

summary_writer = SummaryWriter(tensorboard_save_dir.absolute().as_posix())


parser = argparse.ArgumentParser('ETC')
# args = parser.parse_args()
args = parser.parse_args(args=[])

args.num_past_frames = 1
args.num_future_frames = 1
# args.encH, args.encW, args.encC = 8, 8, 528 # Original setting
# args.encH, args.encW, args.encC = 4, 4, 16 # Increased dimensions for 512 x 512 images
# args.encH, args.encW, args.encC = 16, 16, 528 # Increased dimensions for 512 x 512 images
args.encH, args.encW, args.encC = 16, 16, 128 # Increased dimensions for 512 x 512 images
args.img_channels = 1 #3 channels for BAIR datset
args.epochs = 10

args.N = 8

args.AE_lr = 2e-4

args.batch_size = args.N
test_past_frames = 1
test_future_frames = 1

import torch
print(torch.__version__)

torch.cuda.is_available()

args.device = torch.device('cuda:0')
print(f"Using {args.device} device")



import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import argparse
from pathlib import Path
import random

# --- SETUP AND ARGS (FOR RUNNABILITY) ---
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(2025)

parser = argparse.ArgumentParser('VPTR Model Args')
args = parser.parse_args(args=[])
# Defining essential arguments for model instantiation
args.N = 8 # Batch size
args.nhead = 8
# CRITICAL LIGHTWEIGHT CHANGE: FFN dimensions drastically reduced
args.d_attn1 = 512 # Used for 128-channel features (Level 3 Attn)
args.d_attn2 = 512 # Used for 256-channel features (Level 4 Attn)
args.img_channels = 3 # <-- CHANGED: Now using 3 channels (GT, Mask, Growth)
args.img_H = 256 # <-- NEW: Height for 256x256 input
args.img_W = 256 # <-- NEW: Width for 256x256 input
args.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BASE_CHANNELS = 32 # Drastically reduced for lightweight design

### Load in GA Data

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split

class Data(Dataset):
    pass

train_data = Data()
val_data = Data()
test_data = Data()

N = 4
Nv = N
batch_size = N
ngpus = 1
num_workers = 1

%cd /Users/Pracioppo/Desktop/VPTR/GA_Zubens_data

FAFs = torch.from_numpy(sio.loadmat('fafs_reg3.mat')["fafs_reg3"].astype(np.float32)).reshape(660,1,4,256,256)
masks = torch.from_numpy(sio.loadmat('masks_reg3.mat')["masks_reg3"].astype(np.float32)).reshape(660,1,4,256,256)

masks /= torch.max(masks)
FAFs /= torch.max(FAFs)

print(FAFs.size())
print(masks.size())


train_masks = masks[0:560]
test_masks = masks[560:]
train_FAFs = FAFs[0:560]
test_FAFs = FAFs[560:]


train_residuals = f_Residuals(train_masks)
test_residuals = f_Residuals(test_masks)

train_residuals = train_residuals - torch.min(train_residuals)
train_residuals = train_residuals/torch.max(train_residuals)

test_residuals = test_residuals - torch.min(test_residuals)
test_residuals = test_residuals/torch.max(test_residuals)

print(train_residuals.size())


args.num_t_steps = 4
img_sz = 256

Train_Data = torch.concatenate((train_FAFs,train_masks,train_residuals),4).to(args.device)
Test_Data = torch.concatenate((test_FAFs,test_masks,test_residuals),4).to(args.device)
All_Data = torch.concatenate((Train_Data, Test_Data))

print(Train_Data.size())
print(Test_Data.size())


# Get sharply defined masks
All_Data[:,:,:,:,256:] = (All_Data[:,:,:,:,256:] > 0.5)



# Get data for K fold cross validation

sz = 70
k = 0

Test_Data = All_Data[k*sz:(k+1)*sz].to(args.device)
Train_Data = torch.cat((All_Data[:k*sz], All_Data[(k+1)*sz:]), axis=0).to(args.device)

print(Test_Data.size())
print(Train_Data.size())

Data = Test_Data
for i in np.arange(1):
    print(i)
    i = 2
    j = 3
    for j in np.arange(4):
        img = Data[i*10,0,j,:,256:512].cpu().detach().numpy()
        plt.imshow(img, cmap='gray')
        plt.show()
        
    for j in np.arange(3):
        residual = (Data[i*10,0,j,:,512:]).cpu().detach().numpy()
        plt.imshow(residual, cmap='gray')
        plt.show()
        
del train_FAFs, train_masks, train_residuals
del test_FAFs, test_masks, test_residuals

Train_Data.size()
# torch.Size([590, 1, 4, 256, 768])

### Net Architectures

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import argparse
from pathlib import Path
import random

# --- SETUP AND ARGS (FOR RUNNABILITY) ---
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(2023)

parser = argparse.ArgumentParser('VPTR Model Args')
args = parser.parse_args(args=[])
# Defining essential arguments for model instantiation
args.N = 8 # Batch size
args.nhead = 8
# CRITICAL LIGHTWEIGHT CHANGE: FFN dimensions drastically reduced
# Level 3 (128 channels) FFN reduced from 512 to 256 (2x expansion)
args.d_attn1 = 192 # <--- FURTHER REDUCED: Used for 128-channel features (Level 3 Attn)
# Level 4 (256 channels) FFN kept at 512 (2x expansion)
args.d_attn2 = 384 # <--- FURTHER REDUCED: Used for 256-channel features (Level 4 Attn)
args.img_channels = 3 # <-- CORRECTED: Setting default input channel to 3 globally
args.img_sz = 256 # CRITICAL: Set to 256 as required
args.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BASE_CHANNELS = 28 # <--- AGGRESSIVE REDUCTION: Down from 32 to 28 for parameter savings

# --- CORE UTILITY MODULES (RoPE IMPLEMENTATION) ---

def rotate_half(x):
    """Rotates the last dimension of the tensor by half."""
    # x shape: (..., D_H)
    D_H = x.shape[-1]
    
    # Reshape x to (..., D_H/2, 2)
    x = x.reshape(*x.shape[:-1], D_H // 2, 2) 
    x1, x2 = x.unbind(dim=-1) # x1, x2 shape: (..., D_H/2)
    
    # Concatenate the rotated parts (-x2, x1) along the last dimension
    rotated_x = torch.cat((-x2, x1), dim=-1) # rotated_x shape: (..., D_H)
    
    # Returns a tensor of the original shape (..., D_H)
    return rotated_x

class RotaryPositionalEmbedding(nn.Module):
    """Generates the sin/cos vectors for RoPE."""
    def __init__(self, dim, max_seq_len=4096):
        super().__init__()
        # Calculate inverse frequency (1/10000^(2i/d))
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq)
        self.max_seq_len = max_seq_len
        
    def forward(self, seq_len, device):
        if seq_len > self.max_seq_len:
            # Resize cache or handle sequence length exceeding max
            pass
            
        t = torch.arange(seq_len, device=self.inv_freq.device, dtype=self.inv_freq.dtype)
        # Outer product to get frequencies: (L, 1) * (1, D/2) -> (L, D/2)
        freqs = torch.einsum("i,j->ij", t, self.inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1) # (L, D) - Repeated sin/cos parts
        
        # pe: (2, L, D) -> [cos, sin]
        pe = torch.stack([emb.cos(), emb.sin()], dim=0).to(device) 
        return pe # Shape (2, L, D)

class RoPEMultiheadAttention(nn.Module):
    """Custom MHA layer that applies RoPE to Q/K after projection."""
    def __init__(self, embed_dim, num_heads, dropout=0., batch_first=True):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.batch_first = batch_first
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"
        
        # Projection for Q, K, V (3*D)
        self.in_proj = nn.Linear(embed_dim, 3 * embed_dim, bias=True) 
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)
        self.scaling = self.head_dim ** -0.5
        
        # RoPE applied per head dimension
        # Max sequence length for 32x32 is 1024. For 64x64 is 4096. Set to 4096 for safety.
        self.rope_emb = RotaryPositionalEmbedding(self.head_dim, max_seq_len=4096) 
        
    def forward(self, query, key, value):
        N, L_q, D = query.shape
        L_k = key.shape[1] # Sequence length of key
        
        is_cross_attn = (query is not key)
        
        # 1. Project Q, K, V
        qkv = self.in_proj(query)
        q, k, v = qkv.chunk(3, dim=-1)
        
        if is_cross_attn:
            # If cross-attention, K and V are derived from the encoder's feature map (key/value input)
            # and Q is derived from the decoder's feature map (query input).
            kv_proj = self.in_proj(key) # key == value for cross-attention in this architecture
            k_proj, v_proj, _ = kv_proj.chunk(3, dim=-1)
            k, v = k_proj, v_proj
            
        # 2. Reshape and split heads: (N, L, D) -> (N, H, L, H_D)
        def reshape_heads(x):
            return x.reshape(N, -1, self.num_heads, self.head_dim).transpose(1, 2)

        q = reshape_heads(q) # (N, H, L_q, H_D)
        k = reshape_heads(k) # (N, H, L_k, H_D)
        v = reshape_heads(v)
        
        # 3. Apply RoPE to Q and K
        seq_len = max(L_q, L_k)
        
        # rot_emb_cos_sin shape: (2, L, H_D)
        rot_emb_cos_sin = self.rope_emb(seq_len, query.device) 
        
        # Unpack cos/sin: (2, L, H_D) -> (1, L, H_D)
        cos, sin = rot_emb_cos_sin.chunk(2, dim=0) 
        
        # Reshape to (1, 1, L, H_D) for broadcasting over (N, H)
        cos = cos.unsqueeze(1) 
        sin = sin.unsqueeze(1) 

        # Apply RoPE (RoPE is element-wise on the last dimension)
        # Indexing cos/sin for L_q and L_k
        q = (q * cos[:, :, :L_q]) + (rotate_half(q) * sin[:, :, :L_q])
        k = (k * cos[:, :, :L_k]) + (rotate_half(k) * sin[:, :, :L_k])
        
        # 4. Scaled Dot-Product Attention
        attn_output_weights = torch.matmul(q * self.scaling, k.transpose(-2, -1)) # (N, H, L_q, L_k)
        
        attn_output_weights = F.softmax(attn_output_weights, dim=-1)
        attn_output_weights = self.dropout(attn_output_weights)
        
        attn_output = torch.matmul(attn_output_weights, v) # (N, H, L_q, H_D)
        
        # 5. Concatenate heads and final projection
        attn_output = attn_output.transpose(1, 2).flatten(start_dim=-2) # (N, L_q, D)
        output = self.out_proj(attn_output)
        
        return output, attn_output_weights.mean(dim=1)

class RoPETransformerEncoderLayer(nn.Module):
    """Custom TransformerEncoderLayer using RoPEMultiheadAttention."""
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0., batch_first=True):
        super().__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.attn = RoPEMultiheadAttention(d_model, nhead, dropout=dropout, batch_first=batch_first)
        self.norm2 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(dim_feedforward, d_model),
            nn.Dropout(dropout)
        )
        self.d_model = d_model

    def forward(self, src):
        # Self-Attention Block
        src_norm = self.norm1(src)
        attn_out, _ = self.attn(src_norm, src_norm, src_norm)
        src = src + attn_out # Residual
        
        # FFN Block
        src_norm = self.norm2(src)
        ffn_out = self.ffn(src_norm)
        src = src + ffn_out # Residual
        
        return src
    
class ConvBlock(nn.Module):
    """Standard Convolutional Block: Conv -> ReLU -> Conv -> ReLU"""
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

# --- ENCODER (U-NET DOWN PATH: CNNs only for high resolution) ---

class Unet_Enc(nn.Module):
    def __init__(self, args, img_channels=1, base_channels=BASE_CHANNELS):
        super(Unet_Enc, self).__init__()
        
        # --- Downsampling Path (5 Levels Total, 4 Skips + 1 Bottleneck) ---
        
        # Level 1 (256x256 -> 128x128)
        # Input: C -> 28
        self.conv1 = ConvBlock(img_channels, base_channels) # 256x256x28 (Skip feats1u)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        # Level 2 (128x128 -> 64x64)
        # Input: 28 -> 56
        self.conv2 = ConvBlock(base_channels, base_channels * 2) # 128x128x56 (Skip feats2u)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        # Level 3 (64x64 -> 32x32)
        # Input: 56 -> 112
        self.conv3 = ConvBlock(base_channels * 2, base_channels * 4) # 64x64x112 (Skip feats3u)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        
        # Level 4 (32x32 -> 16x16)
        # Input: 112 -> 224
        self.conv4 = ConvBlock(base_channels * 4, base_channels * 8) # 32x32x224 (Skip feats4u)
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        
        # Attention 1 (Applied on Level 4: 32x32 features, 224 channels)
        self.te1_model_dim = base_channels * 8 # 224 channels
        # ONLY ONE Self-Attention Layer
        self.te_sa4_1 = RoPETransformerEncoderLayer(d_model=self.te1_model_dim, nhead=args.nhead, dim_feedforward=args.d_attn2, dropout=0, batch_first=True)
        # self.te_sa4_2 removed for parameter reduction
        
        # Level 5 (16x16): Bottleneck
        # Input: 224 -> 448
        self.conv5 = ConvBlock(base_channels * 8, base_channels * 16) # 16x16x448 (Bottleneck_4d)

    def forward(self, in_frames):
        N = in_frames.size(0)
        
        # Level 1: 256x256x28 (Pure CNN)
        feats1u = self.conv1(in_frames) 
        x = self.pool1(feats1u)
        
        # Level 2: 128x128x56 (Pure CNN)
        feats2u = self.conv2(x)
        x = self.pool2(feats2u)
        
        # Level 3: 64x64x112 (Pure CNN)
        feats3u = self.conv3(x)
        x_pre_attn = self.pool3(feats3u) # 32x32x112
        
        # Level 4: 32x32x224 (CNN + Attention)
        feats4u_pre = self.conv4(x_pre_attn)
        
        # Attention 1 (Self-Attention on L4 features)
        tokens4 = feats4u_pre.flatten(2).transpose(1, 2) # (N, 1024, 224)
        
        # Single Self-Attention Layer
        q1 = self.te_sa4_1(tokens4)
        # q2 removed
        
        # Fuse Attention output with residual
        q1_4d = q1.transpose(1, 2).reshape(N, self.te1_model_dim, *x_pre_attn.shape[2:])
        feats4u = feats4u_pre + q1_4d # Skip feature 4
        
        # Bottleneck (L5: 16x16x448)
        x_bottleneck_pre = self.pool4(feats4u) # 16x16x224
        bottleneck_4d = self.conv5(x_bottleneck_pre) # 16x16x448
        
        # Returns 4 skips + 4D bottleneck feature
        return feats1u, feats2u, feats3u, feats4u, bottleneck_4d

    
# --- DECODER (U-NET UP PATH: Attention in low resolution, CNNs for high) ---

class Unet_Dec(nn.Module):
    def __init__(self, args, img_channels=1, base_channels=BASE_CHANNELS):
        super(Unet_Dec, self).__init__()
        
        # --- Upsampling Path ---
        
        # Level 5 to 4 Up: 16x16x448 -> 32x32x224 (Interact with the L4 skip)
        self.upconv4 = nn.ConvTranspose2d(base_channels * 16, base_channels * 8, kernel_size=2, stride=2)
        # Concat input: 224 (upconv) + 224 (skip feats4u) = 448
        self.up_conv4 = ConvBlock(base_channels * 8 + base_channels * 8, base_channels * 8) # 448 in -> 224 out
        
        # --- Interleaved Attention Layers (Level 4: 32x32x224) ---
        
        self.te2d_model_dim = base_channels * 8 # 224
        # ONLY ONE Self-Attention Layer
        self.te_sa4d_1 = RoPETransformerEncoderLayer(d_model=self.te2d_model_dim, nhead=args.nhead, dim_feedforward=args.d_attn2, dropout=0, batch_first=True)
        # self.te_sa4d_2 removed for parameter reduction
        # Cross-Attention (RoPE)
        self.mha4_ca = RoPEMultiheadAttention(embed_dim=self.te2d_model_dim, num_heads=args.nhead, dropout=0, batch_first=True) 
        
        # Level 4 to 3 Up: 32x32x224 -> 64x64x112 (Interact with feats3u)
        self.upconv3 = nn.ConvTranspose2d(base_channels * 8, base_channels * 4, kernel_size=2, stride=2)
        # Concat input: 112 (upconv) + 112 (skip feats3u) = 224
        self.up_conv3 = ConvBlock(base_channels * 4 + base_channels * 4, base_channels * 4) # 224 in -> 112 out

        # --- Interleaved Attention Layers (Level 3: 64x64x112) ---
        
        self.te1d_model_dim = base_channels * 4 # 112
        # ONLY ONE Self-Attention Layer
        self.te_sa3d_1 = RoPETransformerEncoderLayer(d_model=self.te1d_model_dim, nhead=args.nhead, dim_feedforward=args.d_attn1, dropout=0, batch_first=True)
        # self.te_sa3d_2 removed for parameter reduction
        # Cross-Attention (RoPE)
        self.mha3_ca = RoPEMultiheadAttention(embed_dim=self.te1d_model_dim, num_heads=args.nhead, dropout=0, batch_first=True) 

        # Level 3 to 2 Up: 64x64x112 -> 128x128x56 (Pure CNN path resumes)
        self.upconv2 = nn.ConvTranspose2d(base_channels * 4, base_channels * 2, kernel_size=2, stride=2)
        # Concat input: 56 (upconv) + 56 (skip feats2u) = 112
        self.up_conv2 = ConvBlock(base_channels * 2 + base_channels * 2, base_channels * 2) # 112 in -> 56 out

        # Level 2 to 1 Up: 128x128x56 -> 256x256x28 (Pure CNN path)
        self.upconv1 = nn.ConvTranspose2d(base_channels * 2, base_channels, kernel_size=2, stride=2)
        # Concat input: 28 (upconv) + 28 (skip feats1u) = 56
        self.up_conv1 = ConvBlock(base_channels + base_channels, base_channels) # 56 in -> 28 out
        
        # Final Output Layer (28 channels -> 3 channel)
        self.final_conv = nn.Conv2d(base_channels, img_channels, kernel_size=1)
        self.sig = nn.Sigmoid()
        
    def forward(self, feats1u_enc, feats2u_enc, feats3u_enc, feats4u_enc, bottleneck_4d):
        N = bottleneck_4d.size(0)
        
        # --- Upsampling Path Start (Bottleneck) ---
        
        # Level 5 to 4 Up: 16x16x448 -> 32x32x224 (Interact with feats4u)
        x = self.upconv4(bottleneck_4d) # x shape: (N, 224, 32, 32)
        x = torch.cat([x, feats4u_enc], dim=1) # (N, 448, 32, 32)
        feats4d = self.up_conv4(x) # (N, 224, 32, 32)
        
        # --- Level 4 Attention (32x32x224) ---
        tokens4d = feats4d.flatten(2).transpose(1, 2) # (N, 1024, 224)
        
        # Single Self-Attention (RoPE)
        b1 = self.te_sa4d_1(tokens4d)
        feats4d += b1.transpose(1, 2).reshape(feats4d.size())
        
        # Cross-Attention (RoPE) (Query uses b1 output, K/V uses Encoder Skip)
        tokens4u = feats4u_enc.flatten(2).transpose(1, 2) # (N, 1024, 224)
        attn_output4, _ = self.mha4_ca(b1, tokens4u, tokens4u) # Q=b1
        feats4d += attn_output4.transpose(1, 2).reshape(feats4d.size())
        
        # Level 4 to 3 Up: 32x32x224 -> 64x64x112 (Interact with feats3u)
        x = self.upconv3(feats4d) # x shape: (N, 112, 64, 64)
        x = torch.cat([x, feats3u_enc], dim=1) # (N, 224, 64, 64)
        feats3d = self.up_conv3(x) # (N, 112, 64, 64)
        
        # --- Level 3 Attention (64x64x112) ---
        tokens3d = feats3d.flatten(2).transpose(1, 2) # (N, 4096, 112)
        
        # Single Self-Attention (RoPE)
        c1 = self.te_sa3d_1(tokens3d)
        feats3d += c1.transpose(1, 2).reshape(feats3d.size())
        
        # Cross-Attention (RoPE)
        tokens3u = feats3u_enc.flatten(2).transpose(1, 2) # (N, 4096, 112)
        attn_output3, _ = self.mha3_ca(c1, tokens3u, tokens3u) # Q=c1
        feats3d += attn_output3.transpose(1, 2).reshape(feats3d.size())

        # Level 3 to 2 Up: 64x64x112 -> 128x128x56 (Pure CNN path resumes)
        x = self.upconv2(feats3d) # x shape: (N, 56, 128, 128)
        x = torch.cat([x, feats2u_enc], dim=1) # (N, 112, 128, 128)
        feats2d = self.up_conv2(x) # (N, 56, 128, 128)

        # Level 2 to 1 Up: 128x128x56 -> 256x256x28 (Pure CNN path)
        x = self.upconv1(feats2d) # x shape: (N, 28, 256, 256)
        x = torch.cat([x, feats1u_enc], dim=1) # (N, 56, 256, 256)
        feats1d = self.up_conv1(x) # (N, 28, 256, 256)
        
        # Final Output
        out = self.final_conv(feats1d)
        out_frames = self.sig(out)

        return out_frames
    
# --- FULL U-NET MODEL WRAPPER ---
class U_Net_Attn(nn.Module):
    """
    A wrapper class combining the Encoder (Unet_Enc) and the Decoder (Unet_Dec)
    to form the complete U-Net with interleaved Rotary Positional Embedding (RoPE) attention.

    The model takes an input frame, encodes it, and then decodes it,
    returning the reconstructed frame along with all intermediate skip connections
    and the 4D bottleneck feature for potential use in auxiliary losses or sequential models.
    """
    def __init__(self, args, img_channels=args.img_channels, base_channels=BASE_CHANNELS):
        super(U_Net_Attn, self).__init__()
        self.E1 = Unet_Enc(args, img_channels, base_channels)
        self.D1 = Unet_Dec(args, img_channels, base_channels)
        
    def forward(self, in_frames):
        # 1. Encoder Pass: Compute 4 skip features and the 4D bottleneck
        feats1u, feats2u, feats3u, feats4u, bottleneck_4d = self.E1(in_frames)
        
        # 2. Decoder Pass: Reconstruct frame using bottleneck and skip connections
        out_frames = self.D1(feats1u, feats2u, feats3u, feats4u, bottleneck_4d)
        
        # Returns reconstructed frame, plus all intermediate features
        return out_frames, feats1u, feats2u, feats3u, feats4u, bottleneck_4d

# --- EXECUTION AND TESTING ---

if __name__ == '__main__':
    print(f"Initializing U_Net_Attn and running a forward pass test on {args.device}...")

    # *** CRITICAL FIX: Set img_channels BEFORE model instantiation ***
    # This ensures the model is built with 3 input/output channels.
    args.img_channels = 3 
    
    # Instantiate the corrected full model
    full_unet = U_Net_Attn(args).to(args.device) 

    # Example Input Test: (N, C, H, W)
    dummy_input = torch.randn(args.N, args.img_channels, args.img_sz, args.img_sz).to(args.device)

    # Forward Pass
    output_frame, f1, f2, f3, f4, btl = full_unet(dummy_input)

    # Count Total Parameters
    num_params_enc = sum(p.numel() for p in full_unet.E1.parameters())
    num_params_dec = sum(p.numel() for p in full_unet.D1.parameters())
    num_params_total = num_params_enc + num_params_dec

    print("\n--- Full Model Summary ---")
    print(f"Total Parameters: {num_params_total:,} (Targeting < 10M)")
    print(f"  Encoder Parameters: {num_params_enc:,}")
    print(f"  Decoder Parameters: {num_params_dec:,}")
    print("\n--- Parameter Breakdown ---")
    
    # Function to print module-specific parameters
    def print_module_params(module, name):
        total = sum(p.numel() for p in module.parameters())
        if total > 0:
            print(f"  {name}: {total:,}")

    # Encoder Breakdown
    print("\nEncoder (Unet_Enc) Component Breakdown:")
    print_module_params(full_unet.E1.conv1, "L1 Conv (3->28)")
    print_module_params(full_unet.E1.conv2, "L2 Conv (28->56)")
    print_module_params(full_unet.E1.conv3, "L3 Conv (56->112)")
    print_module_params(full_unet.E1.conv4, "L4 Conv (112->224)")
    print_module_params(full_unet.E1.te_sa4_1, "L4 Self-Attn (224D)")
    print_module_params(full_unet.E1.conv5, "L5 Bottleneck (224->448)")
    
    # Decoder Breakdown
    print("\nDecoder (Unet_Dec) Component Breakdown:")
    print_module_params(full_unet.D1.upconv4, "L5->L4 UpConv (448->224)")
    print_module_params(full_unet.D1.up_conv4, "L4 Concat Conv (448->224)")
    print_module_params(full_unet.D1.te_sa4d_1, "L4 Self-Attn (224D)")
    print_module_params(full_unet.D1.mha4_ca, "L4 Cross-Attn (224D)")
    
    print_module_params(full_unet.D1.upconv3, "L4->L3 UpConv (224->112)")
    print_module_params(full_unet.D1.up_conv3, "L3 Concat Conv (224->112)")
    print_module_params(full_unet.D1.te_sa3d_1, "L3 Self-Attn (112D)")
    print_module_params(full_unet.D1.mha3_ca, "L3 Cross-Attn (112D)")
    
    print_module_params(full_unet.D1.upconv2, "L3->L2 UpConv (112->56)")
    print_module_params(full_unet.D1.up_conv2, "L2 Concat Conv (112->56)")
    print_module_params(full_unet.D1.upconv1, "L2->L1 UpConv (56->28)")
    print_module_params(full_unet.D1.up_conv1, "L1 Concat Conv (56->28)")
    print_module_params(full_unet.D1.final_conv, "L1 Final Conv (28->3)")


    print("\n--- Test Results ---")
    print(f"Output Frame Shape: {output_frame.shape}")
    print(f"Intermediate Feature Shapes:")
    print(f"  L1 Skip: {f1.shape}")
    print(f"  L2 Skip: {f2.shape}")
    print(f"  L3 Skip: {f3.shape}")
    print(f"  L4 Skip: {f4.shape}")
    print(f"  Bottleneck: {btl.shape}")
    print("Forward pass test completed successfully with U_Net_Attn.")


### Training Functionality

In [ ]:
# --- LOSS FUNCTIONS ---

def dsc(y_true, y_pred):
    """
    Dice Similarity Coefficient (DSC) component.
    Used for binary segmentation or reconstruction tasks to measure overlap.
    """
    smooth = 1.
    y_true_f = torch.flatten(y_true)
    y_pred_f = torch.flatten(y_pred)
    # This calculation implements a soft mask based on the product
    mask = y_true_f * y_pred_f
    intersection = torch.sum(mask)
    score = (2. * intersection + smooth) / (torch.sum(y_true_f) + torch.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    """
    Dice Loss combined with Binary Cross-Entropy (BCE).
    This combined loss stabilizes training for tasks involving binary prediction/reconstruction.
    The primary loss term is (1 - DSC).
    """
    # Dice Loss: 1 - DSC
    dice_l = (1 - dsc(y_true, y_pred))
    # Combined with BCE for stable gradient signals
    bce_l = nn.functional.binary_cross_entropy(y_pred, y_true)
    return dice_l + bce_l


# --- TRAINING FUNCTIONS ---

# --- 2. TRAINING FUNCTIONS (ADJUSTED FOR DATA SIZE) ---

def single_iter_AE(model, optimizer, loss_fn, loss_fn2, in_frames_np, args):
    """
    Runs a single training iteration (batch update) for the Autoencoder (AE) model.
    Handles slicing the 768-wide data tensor to get the 256-wide FAF input.
    """
    model.train()
    
    # 1. Prepare Data
    # in_frames_np shape: (Batch_Size, 1, T=4, H=256, W_total=768)
    batch_size, _, time_steps, H, W_total = in_frames_np.shape
    img_sz = args.img_sz
    
    # Slice FAFs (first 256 columns) and reshape for U-Net input (B*T, C, H, W)
    in_frames_sliced = in_frames_np[:, 0, :, :, 0:img_sz] 
    in_frames = in_frames_sliced.reshape(batch_size * time_steps, 1, H, img_sz)
    
    # Convert to tensor and move to device
    # NOTE: This is safe because in_frames_np was created from a CPU numpy array by DataWrapper
    in_frames = torch.from_numpy(in_frames).float().to(args.device)
    target_frames = in_frames # Autoencoder target is the input

    # 2. Zero Gradients
    optimizer.zero_grad()
    
    # 3. Forward Pass (AE_model returns 6 outputs)
    out_frames, _, _, _, _, _ = model(in_frames)

    # 4. Compute Loss (loss_fn=BCELoss, loss_fn2=L1Loss)
    loss_main = loss_fn(out_frames, target_frames)
    loss_aux = loss_fn2(out_frames, target_frames)
    total_loss = loss_main + loss_aux 

    # 5. Backward Pass and Optimization
    total_loss.backward()
    optimizer.step()

    # 6. Return Loss
    return total_loss.item()

def f_single_epoch_AE(train_data, model, optimizer, loss, loss2, args):
    """ Executes a single training epoch for the Autoencoder model. """
    model.train()
    random_idxs = np.arange(len(train_data.data))
    np.random.shuffle(random_idxs)
    train_data_shuffle = train_data.data[random_idxs]
    
    num_samples = len(train_data.data)
    batch_size = args.batch_size
    args.num_batches = num_samples // batch_size # Set num_batches dynamically

    epoch_losses = np.zeros(args.num_batches)

    for it in tqdm(range(args.num_batches), desc="Batch progress"):
        # Data shape: [Batch_Size, 1, T=4, H=256, W_total=768] (numpy)
        in_frames_np = train_data_shuffle[it * batch_size:(it + 1) * batch_size, :, :, :, :]

        # Call the single iteration function
        epoch_losses[it] = single_iter_AE(model, optimizer, loss, loss2, in_frames_np, args)
        
    return epoch_losses

## Display

def f_display_autoencoder(train_data, model, args, i=0):
    
    random_idxs = np.arange(len(train_data.data))
    np.random.shuffle(random_idxs)
    train_data_shuffle = train_data.data[random_idxs]
    it = 0
    in_frames = train_data_shuffle[it*args.batch_size:(it+1)*args.batch_size,:,:,:,:]

    with torch.no_grad():
        rec_frames, _, _, _, _ = model(in_frames)
        
    if i == 0:
        plt.imshow(in_frames[i,0,0,:,:].detach().cpu().numpy(), cmap='gray')
        plt.show()
        plt.imshow(rec_frames[i,0,0,:,:].detach().cpu().numpy(), cmap='gray')
        plt.show()
    else:
        for i in np.arange(args.batch_size):
            plt.imshow(in_frames[i,0,0,:,:].detach().cpu().numpy(), cmap='gray')
            plt.show()
            plt.imshow(rec_frames[i,0,0,:,:].detach().cpu().numpy(), cmap='gray')
            plt.show()
            
            
# --- 6. TRAINING EXECUTION ---

train_data = DataWrapper(Train_Data.detach().cpu())

# Create necessary path for checkpointing
resume_AE_ckpt.mkdir(exist_ok=True)
print(f"\nCheckpoints will be saved to: {resume_AE_ckpt.resolve()}")

# Model Initialization
AE_model = U_Net_Attn(args).to(args.device)

# Training setup
log_mean_epoch_losses = np.zeros(args.num_epochs)

# Loss functions from your provided script
loss = nn.BCELoss(reduction='mean') # BCE on its own
loss2 = nn.L1Loss(reduction='mean') # L1 on its own

lr = 1E-3
optimizer = torch.optim.Adam(AE_model.parameters(), lr=lr, betas=(0.9, 0.999))

print(f"\nStarting training on device: {args.device} for {args.num_epochs} epochs...")

# Training loop
for epoch in tqdm(np.arange(args.num_epochs), desc="Training progress..."):
    # Pass the DataWrapper instance
    epoch_losses = f_single_epoch_AE(train_data, AE_model, optimizer, loss, loss2, args)
    
    # Calculate and log the mean epoch loss
    mean_epoch_loss = np.mean(epoch_losses)
    log_mean_epoch_losses[epoch] = np.log(mean_epoch_loss)
    
    # Visualization and Plotting (at set intervals)
    if np.mod(epoch, args.show_example_epochs) == 0:
        print(f"\n--- Epoch {epoch}: Mean Loss (log): {log_mean_epoch_losses[epoch]:.4f} ---")
        f_display_autoencoder(train_data, AE_model, args)
        
        plt.figure()
        plt.plot(log_mean_epoch_losses[0:epoch])
        plt.title('Log Mean Epoch Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Log Loss')
        plt.grid()
        plt.show()
        
print("\nTraining complete. Saving final checkpoints...")

# # Final Checkpoint Saving (Adapted to your model structure)
# F_Enc_path_save = resume_AE_ckpt.joinpath('U_F_Enc_c16d4_ckpt1.pth')
# F_Dec_path_save = resume_AE_ckpt.joinpath('U_F_Dec_c16d4_ckpt1.pth')

# # Save the full Unet_Enc and Unet_Dec modules
# torch.save(AE_model.E1.state_dict(), F_Enc_path_save)
# torch.save(AE_model.D1.state_dict(), F_Dec_path_save)

# print(f"Checkpoints saved: {F_Enc_path_save.name} and {F_Dec_path_save.name}")
# print("Script finished execution.")

In [ ]:
def f_single_epoch_FM(Train_Data, model, optimizer, loss, loss2, args):
    
    random_idxs = np.arange(len(Train_Data))
    np.random.shuffle(random_idxs)
    train_data_shuffle = Train_Data[random_idxs]

    epoch_losses = np.zeros(args.num_batches)

    for it in tqdm(range(args.num_batches)):

        in_frames_tot = train_data_shuffle[it*args.batch_size:(it+1)*args.batch_size,:,:,:,:]

        epoch_losses[it] = single_iter_FM(model, optimizer, loss, loss2, in_frames_tot, args)
        
    return epoch_losses

In [ ]:
def f_display(Data, R1, args):
    
    random_idxs = np.arange(len(Data))
#     np.random.shuffle(random_idxs)
    train_data_shuffle = Data[random_idxs]
    
    
    save_folder = 'C:/Users/Pracioppo/Desktop/Example GA Imgs/'
    
    for it in range(1):
#     for it in range(100):
        in_frames_tot = train_data_shuffle[it*args.batch_size:(it+1)*args.batch_size,:,:,:,:]

        with torch.no_grad():

    #         in_frames = in_frames_tot[:,:,0,:,:].unsqueeze(1)
    #         out_frames_tot, _, _, _, _ = R1(in_frames)
            out_frames_tot, _, _, _, _, _ = R1(in_frames_tot)


        for i in range(in_frames_tot.size()[0]):
            plt.figure(figsize=(10,6))
            
            plt.subplot(1, 3, 1)
    #         plt.imshow(in_frames_tot[i,0,j,:,:].detach().cpu().numpy(), cmap='gray')
            plt.imshow(in_frames_tot[i,0,-1,:,:-512].detach().cpu().numpy(), cmap='gray')
            plt.savefig(save_folder + 'raw_image_' + str(it + i) + '.png', bbox_inches='tight', pad_inches=0)

            plt.subplot(1, 3, 2)
#             plt.imshow(in_frames_tot[i,0,j,:,:].detach().cpu().numpy(), cmap='gray')
            plt.imshow((in_frames_tot[i,0,-1,:,-256:]).detach().cpu().numpy(), cmap='gray')
#             plt.imshow((in_frames_tot[i,0,-1,:,-256:]>0.5).detach().cpu().numpy(), cmap='gray')
#             plt.savefig(save_folder + 'growth_ground_truth_' + str(it + i) + '.png', bbox_inches='tight', pad_inches=0)

            plt.subplot(1, 3, 3)
            plt.imshow(out_frames_tot[i,j,0,0,:,:].detach().cpu().numpy(), cmap='gray')
            plt.imshow((out_frames_tot[i,-1,0,0,:,-256:]).detach().cpu().numpy(), cmap='gray')
#             plt.imshow((out_frames_tot[i,-1,0,0,:,-256:]>0.5).detach().cpu().numpy(), cmap='gray')
            plt.savefig(save_folder + 'growth_predicted_' + str(it + i) + '.png', bbox_inches='tight', pad_inches=0)

            plt.show()

            print(out_frames_tot.size())

            print(out_frames_tot.size())

In [ ]:
def f_display2(Data, R1, args):

    train_data_shuffle = Data
    
    save_folder = 'C:/Users/Pracioppo/Desktop/Example GA Imgs/'

    for it in range(100):
#     for it in range(100):
        in_frames_tot = train_data_shuffle[it*args.batch_size:(it+1)*args.batch_size,:,:,:,:]

        with torch.no_grad():

    #         in_frames = in_frames_tot[:,:,0,:,:].unsqueeze(1)
    #         out_frames_tot, _, _, _, _ = R1(in_frames)
            out_frames_tot, _, _, _, _, _ = R1(in_frames_tot)

#         for i in range(in_frames_tot.size()[0]):
#             for i in range(3):
            i = it
            for j in range(4):
#             j = -1
    #             plt.figure(figsize=(10,6))

    #             plt.subplot(1, 3, 1)
        #         plt.imshow(in_frames_tot[i,0,j,:,:].detach().cpu().numpy(), cmap='gray')
    #             plt.imshow(in_frames_tot[i,0,-1,:,:-512].detach().cpu().numpy(), cmap='gray')
    #             plt.savefig(save_folder + 'raw_image_' + str(it + i) + '.png', bbox_inches='tight', pad_inches=0)
                plt.imshow((in_frames_tot[i,0,j,:,-512:-256]>0.5).detach().cpu().numpy(), cmap='gray')
                plt.savefig(save_folder + 'mask_ground_truth_' + str(it + i) + '_step' + str(j) + '.png', bbox_inches='tight', pad_inches=0)

    #             plt.subplot(1, 3, 2)
    #             plt.imshow(in_frames_tot[i,0,j,:,:].detach().cpu().numpy(), cmap='gray')
#                     plt.imshow((in_frames_tot[i,0,j,:,-256:]).detach().cpu().numpy(), cmap='gray')
                plt.imshow((in_frames_tot[i,0,-1,:,-256:]>0.5).detach().cpu().numpy(), cmap='gray')
                plt.savefig(save_folder + 'growth_ground_truth_' + str(it + i) + '_step' + str(j) + '.png', bbox_inches='tight', pad_inches=0)

#                     plt.imshow((out_frames_tot[i,j,0,0,:,-512:-256]).detach().cpu().numpy(), cmap='gray')
                plt.imshow((out_frames_tot[i,-1,0,0,:,-512:-256:]>0.5).detach().cpu().numpy(), cmap='gray')
                plt.savefig(save_folder + 'mask_predicted_' + str(it + i) + '_step' + str(j) + '.png', bbox_inches='tight', pad_inches=0)

    #             plt.subplot(1, 3, 3)
    #             plt.imshow(out_frames_tot[i,j,0,0,:,:].detach().cpu().numpy(), cmap='gray')
#                     plt.imshow((out_frames_tot[i,j,0,0,:,-256:]).detach().cpu().numpy(), cmap='gray')
                plt.imshow((out_frames_tot[i,-1,0,0,:,-256:]>0.5).detach().cpu().numpy(), cmap='gray')
                plt.savefig(save_folder + 'growth_predicted_' + str(it + i) + '_step' + str(j) + '.png', bbox_inches='tight', pad_inches=0)

#                 plt.show()

            print(out_frames_tot.size())

            print(out_frames_tot.size())

In [ ]:
def f_display_all(Data, R1, args):
    
    random_idxs = np.arange(len(Data))
    np.random.shuffle(random_idxs)
    train_data_shuffle = Data[random_idxs]
    
    for it in np.arange(int(len(train_data_shuffle)/args.batch_size)):
    
        in_frames_tot = train_data_shuffle[it*args.batch_size:(it+1)*args.batch_size,:,:,:,:]

        with torch.no_grad():

            out_frames_tot, _, _, _, _, _ = R1(in_frames_tot)

        for i in np.arange(args.batch_size):
#             for j in np.arange(4):
            j = 3

            plt.figure(figsize=(10,6))

            plt.subplot(1, 2, 1)
            plt.imshow(in_frames_tot[i,0,j,:,:].detach().cpu().numpy(), cmap='gray')

            plt.subplot(1, 2, 2)
            plt.imshow(out_frames_tot[i,j,0,0,:,:].detach().cpu().numpy(), cmap='gray')
            plt.show()

In [ ]:
args.d_attn1 = 64
args.d_attn2 = 64
args.d_attn1t = args.d_attn1*3
args.d_attn2t = args.d_attn2*3
args.nhead = 4

U1 = U_Net_F(args)

In [ ]:
random_idxs = np.arange(len(Train_Data))
np.random.shuffle(random_idxs)
train_data_shuffle = Train_Data[random_idxs]
# it = 0
# in_frames_tot = train_data_shuffle[it*args.batch_size:(it+1)*args.batch_size,:,:,:,:]
# in_frames_auto = in_frames_tot.flatten(0,2).unsqueeze(1).unsqueeze(2)
in_frames_auto = train_data_shuffle.flatten(0,2).unsqueeze(1).unsqueeze(2)

train_data.data = in_frames_auto
args.num_batches = int(len(Train_Data)/args.N)

print(train_data.data.size())
print(args.num_batches)

In [ ]:
Train_Data.data.size()

In [ ]:
#####################Training loop (for autoencoder) ###########################

args.num_epochs = 200
args.show_example_epochs = 5

log_mean_epoch_losses = np.zeros(args.num_epochs)

loss = nn.BCELoss() # <-- Use this one
loss2 = nn.L1Loss()

lr = 1E-3
optimizer = torch.optim.Adam(AE_model.parameters(), lr=lr, betas=(0.9, 0.999))

# Train for a maximum of max_epochs:
for epoch in tqdm(np.arange(args.num_epochs), desc="Training progress..."):
    epoch_losses = f_single_epoch_AE(train_data, AE_model, optimizer, loss, loss2, args)
    log_mean_epoch_losses[epoch] = np.log(np.mean(epoch_losses))
#     coord_errors[epoch] = f_coord_error(train_data,train_data_traj,VPTR_Enc, C1, args)
    
    if np.mod(epoch,args.show_example_epochs) == 0:
        f_display_autoencoder(train_data, AE_model, args)
        
        plt.plot(log_mean_epoch_losses[0:epoch])
        plt.show()
        
#         plt.plot(coord_errors[0:epoch])
#         plt.grid()
#         plt.show()

F_Enc_path_save = resume_AE_ckpt.joinpath('U_F_Enc_c16d4_ckpt1')
F_Dec_path_save = resume_AE_ckpt.joinpath('U_F_Dec_c16d4_ckpt1')
Enc_path_save = resume_AE_ckpt.joinpath('U_Enc_c16d4_ckpt1')
Dec_path_save = resume_AE_ckpt.joinpath('U_Dec_c16d4_ckpt1')

torch.save(AE_model.E1.state_dict(), F_Enc_path_save)
torch.save(AE_model.D1.state_dict(), F_Dec_path_save)
torch.save(AE_model.E1.Enc1.state_dict(), Enc_path_save)
torch.save(AE_model.D1.Dec1.state_dict(), Dec_path_save)

In [ ]:
# F_Enc_path_save = resume_AE_ckpt.joinpath('U_F_Enc_c16d4_ckpt1')
# F_Dec_path_save = resume_AE_ckpt.joinpath('U_F_Dec_c16d4_ckpt1')
# Enc_path_save = resume_AE_ckpt.joinpath('U_Enc_c16d4_ckpt1')
# Dec_path_save = resume_AE_ckpt.joinpath('U_Dec_c16d4_ckpt1')

# torch.save(AE_model.E1.state_dict(), F_Enc_path_save)
# torch.save(AE_model.D1.state_dict(), F_Dec_path_save)
# torch.save(AE_model.E1.Enc1.state_dict(), Enc_path_save)
# torch.save(AE_model.D1.Dec1.state_dict(), Dec_path_save)

## Multiple time steps

In [ ]:
def causal_mask(seq_len, device):
    # True where masking should happen
    mask = torch.triu(torch.ones(seq_len, seq_len, device=device), diagonal=1)
    mask = mask.masked_fill(mask == 1, float('-inf'))
    return mask  # shape (seq_len, seq_len)

In [ ]:
def f_encode(feats1u,args):
    feats1u_a = feats1u.flatten(start_dim=2)
    feats1u_b = torch.swapaxes(feats1u_a,1,2)
    feats1u_c = f_pos_encoding_additive(feats1u_b,args).squeeze()
    return feats1u_c

In [ ]:
class Attn_t(nn.Module):
    def __init__(self, dim, args):
        super(Attn_t, self).__init__()

        self.te1 = torch.nn.TransformerEncoderLayer(d_model=dim, nhead=args.nhead, dim_feedforward=dim, dropout=0, layer_norm_eps=1e-05, batch_first=True).to(args.device)
        self.te2 = torch.nn.TransformerEncoderLayer(d_model=dim, nhead=args.nhead, dim_feedforward=dim, dropout=0, layer_norm_eps=1e-05, batch_first=True).to(args.device)

    def forward(self, feats1n_tot):
        
        feats1n_tota = torch.swapaxes(feats1n_tot,1,2)
        sz = feats1n_tota.size()
        x = feats1n_tota.reshape(sz[0],sz[1],sz[2]*sz[3]*sz[4])
        xa = torch.swapaxes(x,1,2)
        xb = f_pos_encoding_additive(xa,args).squeeze()
        
        q1 = self.te1(xb)
        
        q2 = self.te2(q1)

        q2 = torch.swapaxes(q2,1,2)
        
        return torch.swapaxes(q2.reshape(feats1n_tota.size()),1,2)

In [ ]:
class Sliding_Window_Attn(nn.Module):
    def __init__(self, dim, args):
        super(Sliding_Window_Attn, self).__init__()
        
        self.A1 = Attn_t(dim, args)
        
    def forward(self, feats1u_tot):
    
        feats1u_out = torch.zeros_like(feats1u_tot)

        for i in np.arange(3):
            x = f_feats_attn(feats1u_tot, self.A1)
            feats1u_out[:,-(i+1),:,:,:] = x[:,-1,:,:,:]
            feats1u_tot = torch.roll(feats1u_tot,1)
            feats1u_tot[:,0,:,:,:] = 0

        return feats1u_out

In [ ]:
def f_feats_attn(feats1n_tot,model):

    feats1n_tota = torch.swapaxes(feats1n_tot,1,2)
    sz = feats1n_tota.size()
    x = feats1n_tota.reshape(sz[0],sz[1],sz[2]*sz[3]*sz[4])
    xa = torch.swapaxes(x,1,2)
    xb = f_pos_encoding_additive(xa,args).squeeze()
    q1 = model.te1(xb)
    q2 = model.te2(q1)
    q2 = torch.swapaxes(q2,1,2)

    return torch.swapaxes(q2.reshape(feats1n_tota.size()),1,2)

In [ ]:
class Cross_t_Attn(nn.Module):
    def __init__(self, dim, args):
        super(Cross_t_Attn, self).__init__()

        self.mha1 = torch.nn.MultiheadAttention(embed_dim=dim, num_heads=args.nhead, batch_first=True).to(args.device)

    def forward(self, feats1, feats2):

        x0_a = feats1.flatten(start_dim=2)
        x1_a = feats2.flatten(start_dim=2)

        x0_b = torch.swapaxes(x0_a,1,2)
        x1_b = torch.swapaxes(x1_a,1,2)
        
        x0_f = f_pos_encoding_additive(x0_b,args).squeeze()
        x1_f = f_pos_encoding_additive(x1_b,args).squeeze()

        attn_output1, _ = self.mha1(x1_f, x0_f, x0_f)
        
        out = attn_output1.reshape(feats1.size())

        return out

In [ ]:
class Cross_Attn(nn.Module):
    def __init__(self, dim, args):
        super(Cross_Attn, self).__init__()
        
#         self.mask = causal_mask(seq_len, args.device)

        self.mha10 = torch.nn.MultiheadAttention(embed_dim=dim, num_heads=args.nhead, batch_first=True).to(args.device)
        self.mha11 = torch.nn.MultiheadAttention(embed_dim=dim, num_heads=args.nhead, batch_first=True).to(args.device)
        self.mha12 = torch.nn.MultiheadAttention(embed_dim=dim, num_heads=args.nhead, batch_first=True).to(args.device)
        self.mha20 = torch.nn.MultiheadAttention(embed_dim=dim, num_heads=args.nhead, batch_first=True).to(args.device)
        self.mha21 = torch.nn.MultiheadAttention(embed_dim=dim, num_heads=args.nhead, batch_first=True).to(args.device)
        self.mha22 = torch.nn.MultiheadAttention(embed_dim=dim, num_heads=args.nhead, batch_first=True).to(args.device)

    def forward(self, feats):
        
        sz = int(feats.size()[-1]/3)

        x0 = feats[:,:,:,0:sz]
        x1 = feats[:,:,:,sz:2*sz]
        x2 = feats[:,:,:,2*sz:3*sz]

        x0_a = x0.flatten(start_dim=2)
        x1_a = x1.flatten(start_dim=2)
        x2_a = x2.flatten(start_dim=2)

        x0_b = torch.swapaxes(x0_a,1,2)
        x1_b = torch.swapaxes(x1_a,1,2)
        x2_b = torch.swapaxes(x2_a,1,2)
        
        x0_f = f_pos_encoding_additive(x0_b,args).squeeze()
        x1_f = f_pos_encoding_additive(x1_b,args).squeeze()
        x2_f = f_pos_encoding_additive(x2_b,args).squeeze()
        
        attn_output10, _ = self.mha10(x1_f, x0_f, x0_f)
        attn_output11, _ = self.mha11(x1_f, x1_f, x1_f)
        attn_output12, _ = self.mha12(x1_f, x2_f, x2_f)

        attn_output20, _ = self.mha20(x2_f, x0_f, x0_f)
        attn_output21, _ = self.mha21(x2_f, x1_f, x1_f)
        attn_output22, _ = self.mha22(x2_f, x2_f, x2_f)
        
        y1_f = attn_output10 + attn_output11 + attn_output12
        z1_f = attn_output20 + attn_output21 + attn_output22 

        out = torch.zeros_like(feats)
        out[:,:,:,sz:2*sz] = y1_f.reshape(x1.size())
        out[:,:,:,2*sz:3*sz] = z1_f.reshape(x1.size())

        return out

In [ ]:
class UnetD(nn.Module):
    def __init__(self, dim, args):
        super(UnetD, self).__init__()

        self.e0 = nn.Conv2d(dim, dim, kernel_size=3, padding=1).to(args.device)
        self.e1 = nn.Conv2d(dim, dim, kernel_size=2, stride = 2, padding=0).to(args.device)
        self.e2 = nn.Conv2d(dim, dim, kernel_size=2, stride = 2, padding=0).to(args.device)
        self.m1 = nn.ConvTranspose2d(dim, dim, kernel_size=2, stride=2).to(args.device)
        self.m2 = nn.ConvTranspose2d(dim, dim, kernel_size=2, stride=2).to(args.device)

        self.ReLU = nn.ReLU()

        self.c1 = torch.nn.Parameter(torch.tensor([0.0])).to(args.device)
        self.c2 = torch.nn.Parameter(torch.tensor([0.0])).to(args.device)

    def forward(self, x):
        x0 = self.ReLU(self.e0(x)) + x
        x1 = self.ReLU(self.e1(x0))
        x2 = self.ReLU(self.e2(x1))
        x3 = self.c1*self.ReLU(self.m1(x2)) + x1
        x4 = self.c2*self.m2(x3)

        return x4

In [ ]:
class LinD(nn.Module):
    def __init__(self, dim):
        super(LinD, self).__init__()
        
        # Decoder
        self.h1 = nn.Linear(dim, dim).to(args.device)
        self.h2 = nn.Linear(dim, dim).to(args.device)
        self.h3 = nn.Linear(dim, dim).to(args.device)
        self.h4 = nn.Linear(dim, dim).to(args.device)
        
        self.ReLU = nn.ReLU()

    def forward(self, x):
        a1 = self.ReLU(self.h1(x))
        a2 = self.ReLU(self.h2(a1))
        a3 = self.ReLU(self.h3(a2))
        a4 = self.h4(a3)

        return a4

In [ ]:
class GatedMultiheadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, device):
        super().__init__()
        self.mha = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        # Learnable scalar gating parameters
        self.w_gamma = nn.Parameter(torch.tensor(1.0, device=device))
        self.b_gamma = nn.Parameter(torch.tensor(0.0, device=device))
        self.sigmoid = nn.Sigmoid()

    def forward(self, query, key, value, attn_mask=None):
        # Standard multi-head attention
        attn_output, attn_weights = self.mha(query, key, value, attn_mask=attn_mask)
        # attn_weights: [batch, seq_len, seq_len], sum along keys gives total precision per query
        row_sums = attn_weights.sum(dim=-1, keepdim=True)  # shape [batch, seq_len, 1]

        # Compute confidence gate
        gamma = self.sigmoid(self.w_gamma * row_sums + self.b_gamma)  # shape [batch, seq_len, 1]

        # Modulate the attention output
        attn_output = gamma * attn_output + query  # residual connection

        return attn_output, attn_weights

    
class GatedTransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, device):
        super().__init__()
        self.attn = GatedMultiheadAttention(embed_dim=d_model, num_heads=nhead, device=device)
        self.norm1 = nn.LayerNorm(d_model)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.0)

    def forward(self, x):
        # x: [batch, seq, dim]
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(attn_out)
        ff_out = self.linear2(torch.relu(self.linear1(x)))
        x = self.norm2(x + ff_out)
        return x


In [ ]:
class DynNet(nn.Module):
    def __init__(self, args):
        super(DynNet, self).__init__()

        self.UD1 = UnetD(128, args)
        self.UD1_s = UnetD(128, args)
        self.CA1 = Cross_Attn(128, args)
        self.CA2 = Cross_Attn(128, args)
        self.CA3 = Cross_Attn(64, args)
        self.CA4 = Cross_Attn(64, args)
        self.UD2 = UnetD(64, args)
        self.UD2_s = UnetD(64, args)
        self.H4 = LinD(64)
        self.H5 = LinD(16)
        
        self.ea = nn.Conv2d(128, 64, kernel_size=3, padding=1).to(args.device) # output: 136x136x256
        self.eb = nn.Conv2d(64, 128, kernel_size=3, padding=1).to(args.device) # output: 136x136x256
        
        self.te1u_1 = torch.nn.TransformerEncoderLayer(d_model=args.d_attn1, nhead=args.nhead, dim_feedforward=args.d_attn1, dropout=0, layer_norm_eps=1e-05, batch_first=True).to(args.device)
#         self.te1u_1 = GatedTransformerEncoderLayer(d_model=args.d_attn1, nhead=args.nhead, dim_feedforward=args.d_attn, args.device)

        # self.te1u_2 = torch.nn.TransformerEncoderLayer(d_model=args.d_attn1, nhead=args.nhead, dim_feedforward=args.d_attn1, dropout=0, layer_norm_eps=1e-05, batch_first=True).to(args.device)

        self.te2u_1 = torch.nn.TransformerEncoderLayer(d_model=args.d_attn2, nhead=args.nhead, dim_feedforward=args.d_attn2, dropout=0, layer_norm_eps=1e-05, batch_first=True).to(args.device)
#         self.te2u_2 = torch.nn.TransformerEncoderLayer(d_model=args.d_attn2, nhead=args.nhead, dim_feedforward=args.d_attn2, dropout=0, layer_norm_eps=1e-05, batch_first=True).to(args.device)
        
    def forward(self, feats1u, feats2u, feats4u_f, feats5_f):

        feats1u_s = torch.zeros_like(feats1u)
        feats1u_s[:,:,:,0:16] = feats1u[:,:,:,0:16]
        feats1u_s[:,:,:,16:32] = feats1u[:,:,:,32:48]
        feats1u_s[:,:,:,32:48] = feats1u[:,:,:,16:32]
        
        feats1u = feats1u + self.CA1(feats1u)
        feats1u_s = feats1u_s + self.CA1(feats1u_s)

        v1 = self.UD1(feats1u)
        v2 = self.UD1_s(feats1u_s)

        V1 = v1 + v2

        feats1u = feats1u + V1
        feats1u[:,:,:,32:48] = feats1u[:,:,:,32:48] + V1[:,:,:,16:32]
#         feats1u[:,:,:,32:48] = V1[:,:,:,16:32]        
        
        feats1u_b = self.ea(feats1u)
        feats1u_a = feats1u_b.flatten(start_dim=2)
        feats1u_a = torch.swapaxes(feats1u_a,1,2)
        feats1u_a = f_pos_encoding_additive(feats1u_a,args).squeeze()
        
#         seq_len = feats1u_a.size()[1]
#         mask = causal_mask(seq_len, args.device)
#         q2 = self.te1u_1(feats1u_a, src_mask=mask, is_causal=True)
        q2 = self.te1u_1(feats1u_a)
        q2 = torch.swapaxes(q2,1,2)
        feats1u_b = feats1u_b + q2.reshape(feats1u_b.size())
        feats1u = feats1u + self.eb(feats1u_b)

        feats2u_s = torch.zeros_like(feats2u)
        feats2u_s[:,:,:,0:4] = feats2u[:,:,:,0:4]
        feats2u_s[:,:,:,4:8] = feats2u[:,:,:,8:12]
        feats2u_s[:,:,:,8:12] = feats2u[:,:,:,4:8]
        
        feats2u = feats2u + self.CA3(feats2u)
        feats2u_s = feats2u_s + self.CA4(feats2u_s)

        v3 = self.UD2(feats2u)
        v4 = self.UD2_s(feats2u_s)

        V2 = v3 + v4

        feats2u = feats2u + V2
        feats2u[:,:,:,8:12] = feats2u[:,:,:,8:12] + V2[:,:,:,4:8]
#         feats2u[:,:,:,8:12] = V2[:,:,:,4:8]

        feats2u_a = feats2u.flatten(start_dim=2)
        feats2u_a = torch.swapaxes(feats2u_a,1,2)
        feats2u_a = f_pos_encoding_additive(feats2u_a,args).squeeze()
        
#         seq_len = feats2u_a.size()[1]
#         mask = causal_mask(seq_len, args.device)
#         q2 = self.te2u_1(feats2u_a, src_mask=mask, is_causal=True)
        q2 = self.te2u_1(feats2u_a)
        q2 = torch.swapaxes(q2,1,2)
        feats2u = feats2u + q2.reshape(feats2u.size())

        feats4u_f = feats4u_f + self.H4(feats4u_f)
        feats5_f = feats5_f + self.H5(feats5_f)

        return feats1u, feats2u, feats4u_f, feats5_f

In [ ]:
class RM_net3(nn.Module):
    def __init__(self, args):
        super(RM_net3, self).__init__()
        
        self.U1 = U_Net_F(args)
        self.F1 = DynNet(args)
        
        self.SWA1 = Sliding_Window_Attn(128,args)
        self.SWA2 = Sliding_Window_Attn(64,args)
        
        self.CtA12_1 = Cross_t_Attn(128,args)
        self.CtA12_2 = Cross_t_Attn(64,args)
        self.CtA13_1 = Cross_t_Attn(128,args)
        self.CtA13_2 = Cross_t_Attn(64,args)

    def forward(self, in_frames_tot):
        
        feats1u_tot = torch.zeros(args.batch_size, 3, 128, 16, 48).to(args.device)
        feats2u_tot = torch.zeros(args.batch_size, 3, 64, 4, 12).to(args.device)
        feats4u_tot = torch.zeros(args.batch_size, 3, 64).to(args.device)
        feats5u_tot = torch.zeros(args.batch_size, 3, 16).to(args.device)
        
        feats1u_tot_p = torch.zeros_like(feats1u_tot).to(args.device)
        feats2u_tot_p = torch.zeros_like(feats2u_tot).to(args.device)
        feats4u_tot_p = torch.zeros_like(feats4u_tot).to(args.device)
        feats5u_tot_p = torch.zeros_like(feats5u_tot).to(args.device)
        
        feats1n_tot = torch.zeros(args.batch_size, 4, 128, 16, 48).to(args.device)
        feats2n_tot = torch.zeros(args.batch_size, 4, 64, 4, 12).to(args.device)
        feats4n_tot = torch.zeros(args.batch_size, 4, 64).to(args.device)
        feats5n_tot = torch.zeros(args.batch_size, 4, 16).to(args.device)
        
        out_frames_tot = torch.swapaxes(torch.zeros_like(in_frames_tot),1,2).unsqueeze(3).to(args.device)
        out_frames1 = torch.swapaxes(torch.zeros_like(in_frames_tot),1,2).unsqueeze(3).to(args.device)
        out_frames_reconstruct = torch.zeros_like(in_frames_tot)

        for i in np.arange(3):
            in_framesi = in_frames_tot[:,:,i,:,:].unsqueeze(1)
            feats1u, feats2u, feats4u_f, feats5_f, x4_sz = self.U1.E1(in_framesi)
            
            feats1u_tot[:,i,:,:,:] = feats1u
            feats2u_tot[:,i,:,:,:] = feats2u
            feats4u_tot[:,i,:] = feats4u_f
            feats5u_tot[:,i,:] = feats5_f
        
        feats1u_tot[:,0,:,:,:] = feats1u_tot[:,0,:,:,:] + self.CtA12_1(feats1u_tot[:,0,:,:,:],torch.zeros_like(feats1u_tot[:,0,:,:,:]))
        feats2u_tot[:,0,:,:,:] = feats2u_tot[:,0,:,:,:] + self.CtA12_2(feats2u_tot[:,0,:,:,:],torch.zeros_like(feats2u_tot[:,0,:,:,:]))
        feats1u_tot[:,0,:,:,:] = feats1u_tot[:,0,:,:,:] + self.CtA13_1(feats1u_tot[:,0,:,:,:],torch.zeros_like(feats1u_tot[:,0,:,:,:]))
        feats2u_tot[:,0,:,:,:] = feats2u_tot[:,0,:,:,:] + self.CtA13_2(feats2u_tot[:,0,:,:,:],torch.zeros_like(feats2u_tot[:,0,:,:,:]))
    
        feats1u_tot[:,1,:,:,:] = feats1u_tot[:,1,:,:,:] + self.CtA12_1(feats1u_tot[:,1,:,:,:],feats1u_tot[:,0,:,:,:])
        feats2u_tot[:,1,:,:,:] = feats2u_tot[:,1,:,:,:] + self.CtA12_2(feats2u_tot[:,1,:,:,:],feats2u_tot[:,0,:,:,:])
        feats1u_tot[:,1,:,:,:] = feats1u_tot[:,1,:,:,:] + self.CtA13_1(feats1u_tot[:,1,:,:,:],torch.zeros_like(feats1u_tot[:,1,:,:,:]))
        feats2u_tot[:,1,:,:,:] = feats2u_tot[:,1,:,:,:] + self.CtA13_2(feats2u_tot[:,1,:,:,:],torch.zeros_like(feats2u_tot[:,1,:,:,:]))
        
        feats1u_tot[:,2,:,:,:] = feats1u_tot[:,2,:,:,:] + self.CtA12_1(feats1u_tot[:,2,:,:,:],feats1u_tot[:,1,:,:,:])
        feats2u_tot[:,2,:,:,:] = feats2u_tot[:,2,:,:,:] + self.CtA12_2(feats2u_tot[:,2,:,:,:],feats2u_tot[:,1,:,:,:])
        feats1u_tot[:,2,:,:,:] = feats1u_tot[:,2,:,:,:] + self.CtA13_1(feats1u_tot[:,2,:,:,:],feats1u_tot[:,0,:,:,:])
        feats2u_tot[:,2,:,:,:] = feats2u_tot[:,2,:,:,:] + self.CtA13_2(feats2u_tot[:,2,:,:,:],feats2u_tot[:,0,:,:,:])
        
        feats1u_tot = feats1u_tot + self.SWA1(feats1u_tot)
        feats2u_tot = feats2u_tot + self.SWA2(feats2u_tot)
        
        for i in np.arange(3):
            
            feats1u = feats1u_tot[:,i,:,:,:]
            feats2u = feats2u_tot[:,i,:,:,:]
            feats4u_f = feats4u_tot[:,i,:]
            feats5_f = feats5u_tot[:,i,:]
            
            feats1u_p, feats2u_p, feats4u_f_p, feats5_f_p = self.F1(feats1u, feats2u, feats4u_f, feats5_f)

            feats1u_tot_p[:,i,:,:,:] = feats1u_p
            feats2u_tot_p[:,i,:,:,:] = feats2u_p
            feats4u_tot_p[:,i,:] = feats4u_f_p
            feats5u_tot_p[:,i,:] = feats5_f_p
            
#             print(out_frames_reconstruct.size())
            estimates, _, _, _, _ = self.U1.D1(feats1u, feats2u, feats4u_f, feats5_f, x4_sz)
#             print(estimates.size())
            out_frames_reconstruct[:,:,i,:,:] = estimates.squeeze(1)
            
            if i < 3:
                out_frames_tot[:,i,:,:,:], feats1n_tot[:,i,:,:,:], feats2n_tot[:,i,:,:,:], feats4n_tot[:,i,:], feats5n_tot[:,i,:] = self.U1.D1(feats1u_p, feats2u_p, feats4u_f_p, feats5_f_p, x4_sz)
        
        return out_frames_tot, out_frames_reconstruct, feats1n_tot, feats2n_tot, feats4n_tot, feats5n_tot, out_frames1[:,1:-1,:,:,:].squeeze()

In [ ]:
class CNN_LSTM_Baseline(nn.Module):
    """
    A fair comparison baseline model using LSTMs for temporal modeling.
    
    This model utilizes the identical U-Net Encoder (U1.E1) and Decoder (U1.D1)
    as the RM_net3 model, replacing the SWA, CtA, and DynNet components
    with standard LSTM layers for recurrent temporal processing. This isolates
    the performance difference to the choice of temporal aggregation (LSTM vs. Attention/SWA).
    
    Note: To maintain a parameter count comparable to the attention-based model, 
    we apply LSTMs to the highly-compressed feature vectors (feats4u_f and feats5_f) 
    at the U-Net bottleneck, as is standard practice for efficiency.
    """
    def __init__(self, args, U1):
        super(CNN_LSTM_Baseline, self).__init__()
        
        # 1. Share the U-Net Encoder and Decoder from the main model
        self.U1 = U1 # U1 must contain E1 (Encoder) and D1 (Decoder)
        self.args = args

        # 2. LSTM Modules for Temporal Processing
        # We replace SWA/CtA/DynNet with standard LSTMs for time-series aggregation.
        # LSTM input size matches the feature dimension. 
        # Hidden size is set equal to input size for parameter efficiency.
        
        # LSTM for the deepest feature vector (feats5u_tot, dim=16)
        # Using 2 layers for comparable complexity to a deep attention stack
        self.lstm5 = nn.LSTM(
            input_size=16, 
            hidden_size=16, 
            num_layers=2, 
            batch_first=True
        ).to(args.device)

        # LSTM for the bottleneck feature vector (feats4u_tot, dim=64)
        self.lstm4 = nn.LSTM(
            input_size=64, 
            hidden_size=64, 
            num_layers=2, 
            batch_first=True
        ).to(args.device)
        
        # Since this model is sequential, we need linear layers to map the LSTM 
        # output (final hidden state) back to the prediction space, simulating the 
        # DynNet's role without its complexity.
        self.linear_pred5 = nn.Linear(16, 16).to(args.device)
        self.linear_pred4 = nn.Linear(64, 64).to(args.device)


    def forward(self, in_frames_tot):
        
        B, C, T, H, W = in_frames_tot.size() # Bx3x3x256x768 (example)
        
        # Tensors to store extracted features from the shared encoder
        # These are structured identically to RM_net3
        feats1u_tot = torch.zeros(B, T, 128, 16, 48).to(self.args.device)
        feats2u_tot = torch.zeros(B, T, 64, 4, 12).to(self.args.device)
        feats4u_tot = torch.zeros(B, T, 64).to(self.args.device)
        feats5u_tot = torch.zeros(B, T, 16).to(self.args.device)

        # 1. Feature Extraction (Shared Encoder)
        # Loop through the T=3 input frames to extract features sequentially
        for i in np.arange(T):
            in_framesi = in_frames_tot[:,:,i,:,:].unsqueeze(1)
            # U1.E1 is the shared Unet_Enc module
            feats1u, feats2u, feats4u_f, feats5_f, x4_sz = self.U1.E1(in_framesi)
            
            # Store the extracted features (no CtA or SWA applied yet)
            feats1u_tot[:,i,:,:,:] = feats1u
            feats2u_tot[:,i,:,:,:] = feats2u
            feats4u_tot[:,i,:] = feats4u_f
            feats5u_tot[:,i,:] = feats5_f

        # 2. Temporal Modeling (LSTM Aggregation)
        # We use the final extracted features as the time sequence input
        
        # LSTM for deepest features (feats5u_tot: B x T x 16)
        lstm_out5, (h_n5, c_n5) = self.lstm5(feats5u_tot)
        # The predicted feature is the output from the final time step
        feats5u_p = self.linear_pred5(lstm_out5[:, -1, :])

        # LSTM for bottleneck features (feats4u_tot: B x T x 64)
        lstm_out4, (h_n4, c_n4) = self.lstm4(feats4u_tot)
        feats4u_p = self.linear_pred4(lstm_out4[:, -1, :])
        
        # For the high-resolution features (feats1u and feats2u), 
        # we will simply use the features from the LAST INPUT frame (t=2) 
        # as the state estimate for the decoder, as LSTMs are not typically used 
        # here without massive parameter cost (ConvLSTMs).
        feats1u_p = feats1u_tot[:, -1, :, :, :]
        feats2u_p = feats2u_tot[:, -1, :, :, :]
        
        # We use the last available x4_sz for the decoder input size
        x4_sz = feats1u.size() 


        # 3. Prediction (Shared Decoder)
        # The baseline only predicts the NEXT frame (t=3), unlike RM_net3 
        # which reconstructs all inputs and predicts the next frame.
        
        # We use the shared decoder D1 with the predicted features
        out_frame_next, _, _, _, _ = self.U1.D1(
            feats1u_p, 
            feats2u_p, 
            feats4u_p, 
            feats5u_p, 
            x4_sz
        )
        
        # To match the output structure of RM_net3 (B x T+1 x C x H x W), we 
        # generate placeholders for the features and the first T frames.
        
        # Placeholders for the reconstructed input frames (used only for loss/visualization in RM_net3)
        # We just use the last input frame as the "reconstructed" T=2 frame
        out_frames_tot = torch.zeros(B, T + 1, *out_frame_next.size()[1:]).to(self.args.device)
        out_frames_tot[:, T, :, :, :] = out_frame_next.squeeze(1) # Predicted next frame is at index T=3

        # Placeholders for feature output sequences
        feats1n_tot = torch.zeros(B, T + 1, 128, 16, 48).to(self.args.device)
        feats2n_tot = torch.zeros(B, T + 1, 64, 4, 12).to(self.args.device)
        feats4n_tot = torch.zeros(B, T + 1, 64).to(self.args.device)
        feats5n_tot = torch.zeros(B, T + 1, 16).to(self.args.device)
        
        # Return the output in a format that mirrors RM_net3
        return out_frames_tot, out_frames_reconstruct, feats1n_tot, feats2n_tot, feats4n_tot, feats5n_tot, None

In [ ]:
# model = RM_net2(args)
model = RM_net3(args)

In [ ]:
# # # from pathlib import Path

resume_AE_ckpt = Path('C:/Users/Pracioppo/Desktop/Dynamics/Models')

# F_Enc_path_load = resume_AE_ckpt.joinpath('U_F_Enc_c16d4_ckpt1')
# F_Dec_path_load = resume_AE_ckpt.joinpath('U_F_Dec_c16d4_ckpt1')
# Enc_path_load = resume_AE_ckpt.joinpath('U_Enc_c16d4_ckpt2')
# Dec_path_load = resume_AE_ckpt.joinpath('U_Dec_c16d4_ckpt2')

# model.U1.E1.load_state_dict(torch.load(F_Enc_path_load))
# model.U1.D1.load_state_dict(torch.load(F_Dec_path_load))
# # model.U1.E1.Enc1.load_state_dict(torch.load(Enc_path_load))
# # model.U1.D1.Dec1.load_state_dict(torch.load(Dec_path_load))

In [ ]:
def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = torch.flatten(y_true)
    y_pred_f = torch.flatten(y_pred)
#     mask = 1.*((y_true_f * y_pred_f)>0)
    mask = y_true_f * y_pred_f
    intersection = torch.sum(mask)
    score = (2. * intersection + smooth) / (torch.sum(y_true_f) + torch.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    return (1 - dsc(y_true, y_pred)) + nn.functional.binary_cross_entropy(y_true, y_pred)

from model import GDL
gdl_loss = GDL(alpha = 1)

In [ ]:
def single_iter_FM(R1, optimizer, loss, loss2, in_frames_tot, args, train_flag = True, w_p = 1):

    optimizer.zero_grad() # Zero out gradients

    out_frames_tot, out_frames_reconstruct, feats1n_tot, feats2n_tot, feats4n_tot, feats5n_tot, out_frames1 = R1(in_frames_tot)

    loss_i = 2*loss(out_frames_tot, in_frames_tot[1:]) # Prediction loss
    
    loss_i += loss(out_frames_reconstruct, in_frames_tot) # Reconstruction loss
#     loss_i = 3*dice_loss(out_frames_toti[:,:,:,-256:], in_frames_toti[:,:,:,-256:])
#     loss_i += dice_loss(out_frames_toti, in_frames_toti)
    
    loss_i.backward()
    optimizer.step()
    
    return loss_i

In [ ]:
random_idxs = np.arange(len(Train_Data))
np.random.shuffle(random_idxs)
train_data_shuffle = Train_Data[random_idxs]

epoch_losses = np.zeros(args.num_batches)

#     for it in tqdm(range(args.num_batches)):
for it in tqdm(range(args.num_batches)):

    in_frames_tot = train_data_shuffle[it*args.batch_size:(it+1)*args.batch_size,:,:,:,:]
    break

out_frames_tot, out_frames_reconstruct, feats1n_tot, feats2n_tot, feats4n_tot, feats5n_tot, out_frames1 = model(in_frames_tot)

print(in_frames_tot.size())
out_frames_tot.size()

## Evaluate

In [ ]:
import cv2
from tabulate import tabulate

In [ ]:
def f_dice_jac(mask1,mask2):

    union = 1*((mask1 + mask2) > 0)

    intersect = (mask1*mask2)

    sum1 = np.sum(mask1) + np.sum(mask2)

    dice = 2*np.sum(intersect)/sum1
    jaccard = np.sum(intersect)/np.sum(union)

    return dice, jaccard, union, np.sum(intersect), sum1

In [ ]:
def f_Tot_Dice(Data,args):

#     random_idxs = np.arange(len(Data))
#     np.random.shuffle(random_idxs)
#     test_data_shuffle = Data[random_idxs]
    test_data_shuffle = Data

    num_batches = int(len(Data)/args.N)

    tot_avg_dice = 0
    
    union_tot = 0
    intersect_tot = 0
    sum_tot = 0
#     for it in tqdm(range(num_batches)):
    for it in range(num_batches):

        in_frames_tot = test_data_shuffle[it*args.batch_size:(it+1)*args.batch_size,:,:,:,:]

        gt_frames = in_frames_tot[:,:,-1,:,:].unsqueeze(1)

        past_frames = in_frames_tot[:,:,-2,:,:].unsqueeze(1)

        with torch.no_grad():
            out_frames_tot, _, _, _, _, _, _ = model(in_frames_tot)

    #     out_frames_tot.size()

        out_frames = out_frames_tot[:,-1,:,:,:,:]
    #     out_frames.size()

        dice_tot = 0
        
        for i in np.arange(args.batch_size):
            frame_p = past_frames[i].squeeze().detach().cpu().numpy()[:,256:512]
            frame_x = gt_frames[i].squeeze().detach().cpu().numpy()[:,256:512]
            frame_y = out_frames[i].squeeze().detach().cpu().numpy()[:,256:512]

            diff1 = frame_x - frame_p
            diff2 = frame_y - frame_p
            mask1 = (diff1 > 0.5)
            mask2 = (diff2 > 0.5)

            dice, jaccard, union, intersect, sum1 = f_dice_jac(mask1,mask2)
        
            dice_tot += dice
            intersect_tot += intersect
            union_tot += union
            sum_tot += sum1
            
        avg_dice = dice_tot/args.batch_size
        tot_avg_dice += avg_dice

    tot_avg_dice /= num_batches
    tot_avg_dice2 = 2*intersect_tot/sum_tot

    return tot_avg_dice, tot_avg_dice2

In [ ]:
Data = Test_Data

avg_dice, avg_dice2 = f_Tot_Dice(Data,args)
print(avg_dice)
print(avg_dice2)

In [ ]:
Train_Data.size()
# Test_Data.size()
args.num_batches = 10

In [ ]:
#####################Training loop (for full model) ###########################

Data = Train_Data
args.training_mode = 1
args.num_batches = int(len(Data)/args.N)

RM_path_save = resume_AE_ckpt.joinpath('RM3_ckpt_1')

args.num_epochs = 500
args.show_example_epochs = 1
args.save_epochs = 1

log_mean_epoch_losses = np.zeros(args.num_epochs)

loss = nn.BCELoss() # <-- Use this one
loss2 = nn.L1Loss()

lr = 1E-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

test_dice_v = []
test_dice_v2 = []

# Train for a maximum of max_epochs:
for epoch in tqdm(np.arange(args.num_epochs), desc="Training progress..."):
    epoch_losses = f_single_epoch_FM(Data, model, optimizer, loss, loss2, args)
    log_mean_epoch_losses[epoch] = np.log(np.mean(epoch_losses))
#     coord_errors[epoch] = f_coord_error(train_data,train_data_traj,VPTR_Enc, C1, args)
    
    if np.mod(epoch,args.show_example_epochs) == 0:
        f_display(Data, model, args)
        
        plt.plot(log_mean_epoch_losses[0:epoch])
        plt.grid()
        plt.show()
        
        test_dice, test_dice2 = f_Tot_Dice(Test_Data,args)
        print(test_dice)
        print(test_dice2)
        test_dice_v.append(test_dice)
        test_dice_v2.append(test_dice2)
        plt.plot(test_dice_v)
        plt.plot(test_dice_v2)
        plt.grid()
        plt.show()

In [ ]:
# f_display(Data, model, args)
f_display2(Data, model, args)

In [ ]:
RM_path_save = resume_AE_ckpt.joinpath('RM3_ckpt_1')
torch.save(model.state_dict(), RM_path_save)

In [ ]:
U1_path_save = resume_AE_ckpt.joinpath('RM3_ckpt1_U1')
F1_path_save = resume_AE_ckpt.joinpath('RM3_ckpt1_F1')

torch.save(model.U1.state_dict(), U1_path_save)
torch.save(model.F1.state_dict(), F1_path_save)